In [23]:
import cv2
import matplotlib.pyplot as plt
from time import sleep
import numpy as np

def show_img(img, size=(15, 15)):
    plt.figure(figsize=size)
    
    if len(img.shape) == 2:
        plt.gray()
        plt.imshow(img)
    else:
        plt.imshow(img[:, :, ::-1])
    plt.show()
    
def play_video(video_path, f, frames=-1, window_name='Video', fps=30, **kwargs):
    vidcap = cv2.VideoCapture(video_path)
    length = vidcap.get(cv2.CAP_PROP_FRAME_COUNT) if frames == -1 else frames
    success,img = vidcap.read()

    old_frame, k, counter = img, -1, 0
    while success and counter != frames and k != ord('q'):
        old_frame = img
        cv2.imshow(window_name, f(img, **kwargs))
            
        sleep(1/fps)
        success, img = vidcap.read()
        k = cv2.waitKey(1)
        counter += 1
        
        pos = int(counter/length * 90)
        print('Playing video: ' + '[' + pos*'o' + '>' + ' '*(90 - pos) + ']', end='\r')
    
    cv2.destroyAllWindows()
    vidcap.release()
    
    return old_frame    

def get_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    

# Colored images as input
def whitefy(img_orig, threshold_diff, threshold_max):
    def is_white(pixel):
        b, g, r = map(int, pj)
        return bool(abs(b - g) < threshold_diff and abs(b - r) < threshold_diff
                     and b > threshold_max)
    
    img, shape = img_orig, img_orig.shape
    new_image = np.zeros(img.shape[:2], dtype=np.uint8)
    for i, pi in enumerate(img):
        for j, pj in enumerate(pi):
            new_image[i, j] = 255 if is_white(pj) else 0
    return new_image

def get_edges_with_diff(img, se_1=2, se_2=2, shape_idx=0, iterations_erode=1, iterations_dilate=1):
    if img.shape == 3:
        img = get_gray(img)

    shape = (cv2.MORPH_ELLIPSE, cv2.MORPH_CROSS, cv2.MORPH_RECT)
    se = cv2.getStructuringElement(shape[shape_idx%len(shape)], (se_1, se_2))
    
    img_prev = cv2.erode(img, se, iterations=iterations_erode)
    img = cv2.dilate(img, se, iterations=iterations_dilate)
    
    return img - img_prev

def calibrate(img, f, args, range_min=0, range_max=255, title='calibrating'):
    cv2.imshow(title, img)
    
    for arg in args:
        cv2.createTrackbar(arg, title, range_min, range_max, lambda x : x)
        
    new_img, k = img, -1
    while k != ord('q'):
        cv2.imshow(title, new_img)
        new_kwargs = {k: v for k,v in ((arg, cv2.getTrackbarPos(arg, title)) for arg in args)}
        new_img = f(img, **new_kwargs)
        k = cv2.waitKey(1)
    cv2.destroyAllWindows()

In [30]:
img = play_video('../samples_offside/sample22.mp4')
# show_img(whitefy(img, 30, 200))

# calibrate(img, whitefy, ('threshold_diff', 'threshold_max'), title='whitefy')
# calibrate(img, cv2.Canny, ('threshold1', 'threshold2'), title='canny')
# calibrate(img, get_edges_with_diff, ('se_1', 'se_2', 'iterations_erode', 'iterations_dilate', 'shape_idx'), 2, 20, title='edges')